In [1]:
# Use federated learning of * (2020) paper mlp to give baseline with
# Zero noise, gaussian noise, and laplace noise

ours2orig = [12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27,
28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 254, 41, 42, 43, 44, 45,
46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64,
65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 85,
86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103,
104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118,
119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 136, 138, 132,
133, 134, 135, 220, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152,
153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167,
168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 185, 186,
187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201,
202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216,
217, 218, 219, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232,
233, 137, 234, 235, 236, 237, 238, 239, 240, 241, 250, 251, 255, 256, 257,
258, 259, 260, 261, 262, 263, 242, 243, 244, 245, 0, 1, 2, 3, 4, 5, 6, 7, 8,
9, 10, 11, 83, 84, 131, 139, 140, 141, 181, 182, 183, 184, 246, 247, 248,
249, 252, 253]

def vec2mat(v):
    a,b = np.triu_indices(264,1)
    m = np.zeros((264,264))
    m[a,b] = v
    return m+m.T

def remap(fc, roimap=ours2orig):
    fc = fc[roimap,:]
    fc = fc[:,roimap]
    return fc

print('Complete')

Complete


In [236]:
sites = ['Detroit', 'Hartford', 'Baltimore', 'Chicago']
test_site = 'Dallas'

train_fcs = [[], [], [], []]
train_labels = [[], [], [], []]

test_fcs = []
test_labels = []

import pickle
import numpy as np

diag_map = {'SZP': 0, 'BPP': 1, 'SADBPP': 2, 'SADDEPP': 2, 'NC': 3, 'SADDEPR': 3, 'SADBPR': 3, 'BPR': 3, 'SZR': 3} 

bsnipdir = '/home/anton/Documents/Tulane/Research/data-sav/anton/cohorts/BSNIP/'
bsnip = pickle.load(open(f'{bsnipdir}/demographics.pkl', 'rb'))

a,b = np.triu_indices(264,1)

for sub in bsnip['Age_cal']:
    parts = sub.split('.')
    diag = bsnip['DXGROUP_1'][sub]
    diag = diag_map[diag]
    if diag != 0 and diag != 3:
        continue
    diag = int(diag == 3)
    p = np.load(f'{bsnipdir}/fc/{sub}_task-unk_fc.npy')
    p = remap(vec2mat(p))[a,b]
    if parts[0] == test_site:
        test_fcs.append(p)
        test_labels.append(diag)
    else:
        for i in range(len(sites)):
            if parts[0] == sites[i]:
                train_fcs[i].append(p)
                train_labels[i].append(diag)
                break

for i in range(len(sites)):
    train_fcs[i] = np.stack(train_fcs[i])
    train_labels[i] = np.array(train_labels[i])
    print(train_fcs[i].shape)
    print(train_labels[i])

test_fcs = np.stack(test_fcs)
test_labels = np.array(test_labels)
print(test_fcs.shape)
print(test_labels)

(84, 34716)
[1 0 0 1 1 0 1 1 1 1 1 1 1 1 0 1 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0 1 0 1 1
 1 0 1 1 1 0 1 1 1 0 1 1 1 1 1 1 0 1 1 1 0 1 0 1 0 1 0 1 0 1 1 0 1 1 1 1 1
 1 0 0 0 1 1 1 1 0 1]
(274, 34716)
[0 1 0 0 1 1 1 1 0 1 1 1 1 1 1 1 1 0 1 0 1 1 0 1 1 1 1 1 1 1 1 1 0 1 0 1 0
 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 0 0 1 1 1 1 0 1 1 1 1 1 1 1 1 0 1 1 1
 1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 0 0 0 1 1 1 1
 1 0 1 1 0 1 0 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 0 1 1 0 1 1 0 1 0 1 1 1 1 1 1
 0 0 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1 0
 1 1 1 1 1 0 1 1 1 1 0 1 1 1 0 1 1 1 1 1 1 0 1 0 1 1 1 0 1 1 1 1 1 1 1 0 1
 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
(226, 34716)
[0 0 1 0 0 1 0 0 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0 1 0 1 1 1 1 1 1 1 1 1 0 1
 1 1 1 1 1 1 1 0 1 0 0 1 1 1 1 0 1 1 1 0 0 0 1 1 0 1 0 0 1 1 1 0 1 1 1 1 1
 1 0 1 0 1 1 1 1 0 0 0 1 1 0 0 0 1 1 1 1 0 1 1 1 1 1 0 0 1 1 0 0 1 1 1 0 1
 1 1 1 0

In [269]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(34716, 100).float().cuda()
        self.fc2 = nn.Linear(100, 2).float().cuda()

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

xtrs = []
ytrs = []
models = []
optims = []
ce = nn.CrossEntropyLoss()

for i in range(len(train_fcs)):
    xtrs.append(torch.from_numpy(train_fcs[i]).float().cuda())
    ytrs.append(torch.from_numpy(train_labels[i]).long().cuda())
    models.append(MLP())
    optims.append(torch.optim.Adam(models[-1].parameters(), lr=1e-4, weight_decay=1e-4))

nepochs = 1000
cperiod = 50

for e in range(nepochs):
    for i in range(len(xtrs)):
        model = models[i]
        optim = optims[i]
        xtr = xtrs[i]
        ytr = ytrs[i]
        optim.zero_grad()
        yhat = model(xtr)
        loss = ce(yhat, ytr)
        loss.backward()
        optim.step()
        if e % cperiod == 0 or e == nepochs-1:
            print(f'{e} {i} {float(loss)}')
            # Combine models
            fc1w = torch.randn(models[0].fc1.weight.shape).float().cuda()*0.01
            fc1b = torch.randn(models[0].fc1.bias.shape).float().cuda()*0.01
            fc2w = torch.randn(models[0].fc2.weight.shape).float().cuda()*0.01
            fc2b = torch.randn(models[0].fc2.bias.shape).float().cuda()*0.01
            with torch.no_grad():
                for model in models:
                    fc1w += model.fc1.weight
                    fc1b += model.fc1.bias
                    fc2w += model.fc2.weight
                    fc2b += model.fc2.bias
                for model in models:
                    model.fc1.weight.data = fc1w/len(models)
                    model.fc1.bias.data = fc1b/len(models)
                    model.fc2.weight.data = fc2w/len(models)
                    model.fc2.bias.data = fc2b/len(models)

0 0 0.6726611852645874
0 1 0.6315030455589294
0 2 0.6058948040008545
0 3 0.5062332153320312
50 0 0.053255289793014526
50 1 0.37248465418815613
50 2 0.503170907497406
50 3 0.328654944896698
100 0 0.02229929342865944
100 1 0.21395546197891235
100 2 0.29174432158470154
100 3 0.1743045598268509
150 0 0.008133761584758759
150 1 0.13004377484321594
150 2 0.19446133077144623
150 3 0.09680790454149246
200 0 0.005346157122403383
200 1 0.0753391683101654
200 2 0.13639172911643982
200 3 0.07270938903093338
250 0 0.0038872049190104008
250 1 0.04553089663386345
250 2 0.10176415741443634
250 3 0.051422812044620514
300 0 0.0029991106130182743
300 1 0.03139394149184227
300 2 0.07454376667737961
300 3 0.033642519265413284
350 0 0.0023672881070524454
350 1 0.021981071680784225
350 2 0.05714274197816849
350 3 0.01990070566534996
400 0 0.0020981840789318085
400 1 0.017680641263723373
400 2 0.044993508607149124
400 3 0.015380839817225933
450 0 0.0017405860126018524
450 1 0.013106334954500198
450 2 0.034577

In [270]:
# Test accuracy

with torch.no_grad():
    xt = torch.from_numpy(test_fcs).float().cuda()
    yhat = models[0](xt).detach().cpu().numpy()
    yhat = np.argmax(yhat, axis=1)
    print(yhat)
    acc = np.mean(yhat == test_labels)
    print(acc)

[1 1 0 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1
 0 0 0 1 1 0 1 1 0 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 0 0 1 1 1 1 1
 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1 0 1 1 0 1 1 1 0 0 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
0.808641975308642


In [70]:
print(fc1b)

tensor([ 1.3771e-02,  6.2454e-03,  1.0287e-02,  1.3161e-02, -9.1774e-04,
        -1.4521e-04, -1.3680e-03,  1.0180e-02, -1.9713e-03,  5.1676e-03,
         1.5829e-02, -6.1571e-05, -4.3426e-03, -2.3820e-09, -4.9416e-07,
        -1.0770e-02, -6.5803e-03,  4.0120e-03,  4.9013e-03, -1.0124e-03,
         1.7830e-02,  1.2079e-02, -3.2292e-03, -1.9480e-03,  6.5770e-03,
         1.3057e-03,  1.1209e-03, -3.5607e-03,  3.7679e-03,  9.5693e-03,
        -1.1271e-06,  2.0765e-02,  9.7491e-03,  4.8480e-03,  1.0211e-03,
        -8.5964e-03, -1.0355e-03, -5.7076e-11, -3.2094e-03,  5.3119e-03,
         4.3591e-03, -3.2896e-06,  1.8530e-02, -2.2193e-03, -6.4112e-03,
         2.0112e-03,  1.0433e-02,  5.2180e-03,  7.3585e-03,  3.3083e-03,
        -1.8837e-03,  1.5994e-02,  2.8345e-06,  8.1006e-04, -1.3398e-03,
        -3.9230e-04,  6.1550e-03, -5.6402e-07,  6.8927e-03, -8.6087e-03,
         9.7504e-03,  1.1939e-02,  1.3183e-02, -8.6083e-03,  2.8098e-04,
         1.5049e-06,  1.5482e-02,  1.3654e-03,  8.3